In [ ]:
%autosave 0

### 0. Dependencies

In [ ]:
# Basics
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid")
%matplotlib inline

# Statistical interference
import scipy.stats
from scipy import stats
from scipy.stats import skewtest
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from scipy.stats import chi2

# Preprocessing
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.compose import make_column_transformer

# Estimators
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.ensemble import GradientBoostingClassifier

# Model selection
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

# metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics

# Other dependencies
from sklearn.pipeline import make_pipeline
from sklearn.exceptions import NotFittedError

# 1. Data Overwiew

Anonymous DataSet.<br>
The underlying phenomenon and data quality are unknown.<br>
Binary target 'Y' is to predicted.<br><br>
Data source:<br>
URL: https://www.kaggle.com/datasets/alimohammedbakhiet/anonymous-data<br>

#### First outlook indicates, that some rows consists of zeros only. (Output 1.1)

In [ ]:
print('Output 1.1')
print('Title: First glance at the data')

df = pd.read_csv('or_data.csv')
df

#### No apparent NaNs found, 1120 samples, 18 columns, various dtypes (Output 1.2)

In [ ]:
print('Output 1.2')
print('Title: Data Information\n')

df.info()

#### Target distribution is imbalanced (Output 1.3)

In [ ]:
print('Output 1.3')
print('Title: Target class distribution value counts (normalized)')

df['Y'].value_counts(normalize=True)

#### Except YEAR and R9, all other variables seems to be floats (Output 1.4)

In [ ]:
print('Output 1.4')
print('Title: No. of uniqe values in columns\n')

for col in df.columns:
    print(col, len(df[col].unique()))

#### Nominal feature YEAR has balanced distribution (Output 1.5).

In [ ]:
print('Output 1.5')
print('Title: Nominal feature YEAR - distribution')

df['YEAR'].value_counts()

#### (Output 1.6) and (Output 1.7) implies, that little information is brought by essentially invariant R9 feature.

In [ ]:
print('Output 1.6')
print('Title: R9 feature value counts')

df['R9'].value_counts()

In [ ]:
print('Output 1.7')
print('Title: Target distribution for R9 != 1')

df.loc[(df['R9'] != 1), 'Y'].value_counts()

# 2. Data Cleaning

This section contains procedures that needs to be executed on both, training and testing samples. Last subsection splits the data.

#### Invariant feature R9 is considered irrelevant and is excluded (Output 2.1)

In [ ]:
# Output 2.1
df = df.drop('R9', axis=1)

## 2.1 Dtype conversion

#### Since all features (except YEAR) are deemed to be floats (Output 1.4), dtype conversion is neccessary for further analysis.<br> However, object dtype data contains separators, what is indicated by (Output 2.1.1).

In [ ]:
print('Output 2.1.1')
print('Title: No. of separators found in object dtype data (erroneous data)\n')

for col in df.columns:
    if df[col].dtype == object:
        print('{} contains {} separators'.format(col, df[col].str.contains(',').sum()))

#### Data containing separators have been removed (Output 2.1.2) and the dtype conversion is performed (Output 2.1.3).<br> Effects are summarized in (Output 2.1.4).

In [ ]:
# Output 2.1.2
for col in df.columns:
    if df[col].dtype == object:
        df = df[~(df[col].str.contains(','))]

In [ ]:
# Output 2.1.3
for col in df.columns:
    if df[col].dtype == object:
        df[col] = df[col].astype('float')

In [ ]:
print('Output 2.1.4')
print('Title: Data Information after dtype conversion\n')

df.info()

## 2.2 Rows containing only zeros

#### As it has been shown by (Output 1.1), the data contains rows of zeros only. <br>These rows have been found and removed from the data (Output 2.2.1). <br>Nine rows have been removed (Output 2.2.2). 

In [ ]:
# Output 2.2.1
temp_df = df.drop(['YEAR', 'Y'], axis=1)
indicies = []
for i in temp_df.index:
    if (temp_df.loc[i].sum() == 0):
        indicies.append(i)
for i in indicies:
    df = df.drop(index=i)

In [ ]:
print('Output 2.2.2')
print('Title: Data shape after excluding rows containing zeros only\n')

df.reset_index(drop=True, inplace=True)
df.shape

## 2.3 Duplicates

#### Number of duplicates found is displayed in (Output 2.3.1). These are evenly distributed in a systematic, maybe artificial, manner (Output 2.3.2).<br> All dupicates have been excluded; number of rows in data shrinked to 878 (Output 2.3.3). Target class distribution imbalance has deepened as a consequence (Output 2.3.4) . 

In [ ]:
print('Output 2.3.1')
print('Title: No. of duplicates')

print(df.duplicated().sum())

In [ ]:
print('Output 2.3.2')
print('Title: Duplicates distribution\n')

temp_df = df.loc[df.duplicated(), :]

m = 0
for i in temp_df.index:
    n = 0
    for j in df.index:
        for col in df.columns:
            if (df.loc[i, col] != df.loc[j, col]):
                break
            m += 1 
        if m == len(df.columns):
            n += 1
            if j != i:
                print('duplicate index: {}'.format(j))
        m = 0
    print('row: {}, no. of duplicates: {}\n'.format(i, n - 1))

In [ ]:
print('Output 2.3.3')
print('Title: Data shape')

df.drop_duplicates(inplace = True)
df.shape

In [ ]:
print('Output 2.3.4')
print('Title: target class distribution')

df.Y.value_counts(normalize=True)

## 2.4 Train/Test sample split

#### Before any statistical interference occurs, independent testing sample is created (Output 2.4.1).

In [ ]:
# Output 2.4.1
y = df['Y']
X = df.drop('Y', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=8, stratify=df['Y'])
df = pd.concat([X_train, y_train], axis=1)

# 3. Exploratory Data Analysis

In this section data is visually inspected, correlation analysis containing statistical interference is performed and multicolinearity within features is examinated. Some features are excluded throughout. 

## 3.1 Distributions

### *Continious features:

#### Wealth of information regarding continious features distributions is shown by (Output 3.1.1). Outliers have been detected. 

In [ ]:
print('Output 3.1.1')
print('Title: Continious features distributions\n')

for col in df.columns:
    if (col == 'YEAR') | (col == 'Y'):
        continue
    print('*****')
    print(col)
    print(df[col].describe())
    print('\n')
    print(skewtest(df[col]))
    
    print('\nAll data density plot:')
    plt.figure(figsize=(16, 6))
    sns.kdeplot(x=df[col])
    plt.xlabel(col, size=30, labelpad=20)
    plt.ylabel('density', size=30, labelpad=20)
    plt.show()
    
    print('\nData density without 10% of most extreme values:')
    temp_df = df[(df[col] > df[col].quantile(0.05)) & (df[col] < df[col].quantile(0.95))]
    plt.figure(figsize=(20, 6))
    sns.kdeplot(x=temp_df[col])
    plt.xlabel(col, size=30, labelpad=20)
    plt.ylabel('density w/o outliers', size=30, labelpad=20)
    plt.show()

#### Target distribution in deciles of continious features is given in (Output 3.1.2). <br>For the sake of interpretability, (Output 3.1.2) as well as (Output 3.1.3) has been computed on subsample of training sample.<br> The subsample has been created by randomly removing such a rows, for which perfect 50/50 target distribution in subsample has been achieved.

In [ ]:
print('Output 3.1.2')
print('Title: Visual examination of the target distribution in deciles of continious features\n')

Y1_set = df[df.Y == 1].sample(df.Y.value_counts()[1] - df.Y.value_counts()[0], random_state=666)
for i in Y1_set.index:
    df.drop(index=i, inplace=True)

for col in df.columns:
    if (col == 'YEAR') | (col == 'Y'):
        continue
    plt.figure()
    fig, axs = plt.subplots(figsize=(20, 7))
    sns.countplot(x=pd.qcut(df[col], 10), hue='Y', data=df)
    plt.xlabel(col, size=30, labelpad=20)
    plt.ylabel('count', size=40, labelpad=20)
    plt.tick_params(axis='x', labelsize=13)
    plt.tick_params(axis='y', labelsize=15)
    plt.legend(['Y = 0', 'Y = 1'], loc='upper center', prop={'size': 25})
    plt.show()

### *Categorical features:

#### Visual examination suggests that target distribution and nominal feature YEAR appear to be independent (Output 3.1.3). 

In [ ]:
print('Output 3.1.3')
print('Title: Target distribution and nominal feature YEAR appear to be independent\n')

plt.figure(figsize=(12, 8))
sns.countplot(x=df['YEAR'], hue='Y', data=df)
plt.xlabel('YEAR', size=30, labelpad=20)
plt.ylabel('count', size=40, labelpad=20)
plt.legend(['Y = 0', 'Y = 1'], loc='upper center', prop={'size': 25})
plt.show()
df = df.append(Y1_set)

#### Chi-square independence test has been employed in order to asses the relationship between target variable Y and feature YEAR (Output 3.1.4). Feature YEAR is found to be irrelevant and therefore has been exluded (Output 3.1.5).

In [ ]:
print('Output 3.1.4')
print('Chi-square independence test between target variable Y and feature YEAR\n')

# contingency table
contingency_table=pd.crosstab(df['Y'],df['YEAR'])
# observed values
Observed_Values = contingency_table.values
# expected values
b=scipy.stats.chi2_contingency(contingency_table)
Expected_Values = b[3]
# degree of freedom
no_of_rows=len(contingency_table.iloc[:,0])
no_of_columns=len(contingency_table.iloc[0,:])
d_of_f=(no_of_rows-1)*(no_of_columns-1)
# significance level 5%
alpha=0.05
# chi-square statistic - χ2
from scipy.stats import chi2
chi_square=sum([(o-e)**2./e for o,e in zip(Observed_Values,Expected_Values)])
chi_square_statistic=chi_square[0]+chi_square[1]
# critical_value
critical_value=chi2.ppf(q=1-alpha,df=d_of_f)
# p-value (probability under the null)
p_value=1-chi2.cdf(x=chi_square_statistic,df=d_of_f)

# summary (no relationship between target and 'YEAR')
print('Significance level: ',alpha)
print('Degree of Freedom: ',d_of_f)
print('chi-square statistic:',chi_square_statistic)
print('critical_value:',critical_value)
print('p-value:',p_value)

In [ ]:
# Output 3.1.5
df = df.drop('YEAR', axis=1)
X_train = X_train.drop('YEAR', axis=1)
X_test = X_test.drop('YEAR', axis=1)

## 3.2 Correlation analysis

#### Broad correlational outlook for methods: Spearman, Pearson and Kendall is displayed by (Output 3.2.1), (Output 3.2.2), (Output 3.2.3), respectively. <br><br>Pearson's linear coefficients visibly differs from rank based Spearman and Kendall's coefficients what suggests nonlinear relationships within variables located in right bottom corner. Also, it does seem that features names might actually be ordered and generally some segmentation of underlying phenomenon is reflected.

In [ ]:
print('Output 3.2.1')
print('Title: Broad correlational outlook; method = Spearman\n')

plt.figure(figsize=(20, 20))
sns.heatmap(df.corr(method='spearman'), annot=True, square=True, cmap='coolwarm')
plt.show()

In [ ]:
print('Output 3.2.2')
print('Title: Broad correlational outlook; method = Pearson\n')

plt.figure(figsize=(20, 20))
sns.heatmap(df.corr(), annot=True, square=True, cmap='coolwarm')
plt.show()

In [ ]:
print('Output 3.2.3')
print('Title: Broad correlational outlook; method = Kendall\n')

plt.figure(figsize=(20, 20))
sns.heatmap(df.corr(method='kendall'), annot=True, square=True, cmap='coolwarm')
plt.show()

### 3.2.1 Statistical interference (target vs feature)

#### Statistical significance of 'target vs feature' corelations is conducted for all three methods in (Output 3.2.1.1). 

In [ ]:
print('Output 3.2.1.1')
print('Title: Statistical interference results and conclusion (target vs feature)\n')
# statistical interference (Spearman corr coefficient)
spearman_list = []

for col in df.columns:
    if col == 'Y':
        continue
    if (scipy.stats.spearmanr(df[['Y', col]]).pvalue < 0.05):
        spearman_list.append(col)
        
print('*Statistically significant variables (Spearman corr):')
print(spearman_list)

# statistical interference (Pearson corr coefficient)

pearson_list = []

for col in df.columns:
    if col == 'Y':
        continue
    if (scipy.stats.pearsonr(df['Y'], df[col])[1] < 0.05):
        pearson_list.append(col)
        
print('\n*Statistically significant variables (Pearson corr):')
print(pearson_list)

# statistical interference (Kendall corr coefficient)

kendall_list = []

for col in df.columns:
    if col == 'Y':
        continue
    tau, p_value = stats.kendalltau(df['Y'], df[col])
    if (p_value < 0.05):
        kendall_list.append(col)       

print('\n*Statistically significant variables (Kendall corr):')
print(kendall_list)

siginificant_features = set(spearman_list + pearson_list + kendall_list)

print('\nSIGNIFICANT FEATURES:')
features = []
for col in df.columns:
    if col == 'Y':
        continue
    for var in siginificant_features:
        if col == var:
            features.append(col)
print(features)

#### Conclusion is, that total of 12 features are included for futher analysis, with 3 features being dropped at this stage (Output 3.2.1.2). 

In [ ]:
# Output 3.2.1.2
variables = []
variables_df = ['Y']

for col in df.columns:
    for feature in features:
        if col == feature:
            variables.append(col)
            variables_df.append(col)

X_train = X_train.loc[:, variables]
X_test = X_test.loc[:, variables]
df = df.loc[:, variables_df]

### 3.2.2 Multicolinearity analysis within features

#### Linear correlations among features are shown by (Output 3.2.2.1).

In [ ]:
print('Output 3.2.2.1')
print('Title: Multicollinearity within features (Pearson)\n')

plt.figure(figsize=(18, 18))
sns.heatmap(df[features].corr(), annot=True, square=True, cmap='coolwarm')
plt.show()

#### Either R1 or R2 can be excluded, according to (Output 3.2.2.1).<br> Since R2 has slightly higher corr coefficients associated (Output 3.2.2.2), R1 is excluded (Output 3.2.2.3). 

In [ ]:
print('Output 3.2.2.2')
print('Title: R1 vs R2 relevance comparison\n')
print('\n          R1 vs R2: scatterplot (outliers excluded)')

temp_df=df[(df['R1'] < 8) & (df['R1'] > -5)]

plt.figure(figsize=(12, 8))
sns.scatterplot(x=temp_df['R1'], y=temp_df['R2'],data=temp_df)
plt.xlabel('R1', size=30, labelpad=20)
plt.ylabel('R2', size=30, labelpad=20)
plt.show()

for col in ['R1', 'R2']:
    print('\n{} vs target'.format(col))
    print('Pearson corr coefficient between {} and target with p_value:'.format(col))
    print(scipy.stats.pearsonr(df['Y'], df[col]))
    print('\n{} vs target'.format(col))
    print('Spearman corr coefficient between {} and target with p_value:'.format(col))
    print(scipy.stats.spearmanr(df[['Y', col]]))
    print('\n{} vs target'.format(col))
    print('Kendall corr coefficient between {} and target with p_value:'.format(col))
    tau, p_value = stats.kendalltau(df['Y'], df[col])
    print('({},{})'.format(tau, p_value))

In [ ]:
# Output 3.2.2.3
df = df.drop('R1', axis=1)
X_train = X_train.drop('R1', axis=1)
X_test = X_test.drop('R1', axis=1)

#### As (Output 3.2.2.1) further indicates, this approach can be repeated with R14 and R16 (Output 3.2.2.4). This is a close call, but arguably R16 is excluded (Output 3.2.2.5).

In [ ]:
print('Output 3.2.2.4')
print('Title: R14 vs R16 relevance comparison\n')
print('\n          R14 vs R16: scatterplot (outliers excluded)')

temp_df=df[(df['R14'] > -1.6) & (df['R16'] < 2) & (df['R14'] < 1.3)]

plt.figure(figsize=(12, 8))
sns.scatterplot(x=temp_df['R14'], y=temp_df['R16'],data=temp_df)
plt.xlabel('R14', size=30, labelpad=20)
plt.ylabel('R16', size=30, labelpad=20)
plt.show()

for col in ['R14', 'R16']:
    if col == 'Y':
        continue
    print('\n{} vs target'.format(col))
    print('Pearson corr coefficient between {} and target with p_value:'.format(col))
    print(scipy.stats.pearsonr(df['Y'], df[col]))
    print('\n{} vs target'.format(col))
    print('Spearman corr coefficient between {} and target with p_value:'.format(col))
    print(scipy.stats.spearmanr(df[['Y', col]]))
    print('\n{} vs target'.format(col))
    print('Kendall corr coefficient between {} and target with p_value:'.format(col))
    tau, p_value = stats.kendalltau(df['Y'], df[col])
    print('({},{})'.format(tau, p_value))

In [ ]:
# Output 3.2.2.5
df = df.drop('R16', axis=1)
X_train = X_train.drop('R16', axis=1)
X_test = X_test.drop('R16', axis=1)

#### Remaining features might convey predictive information and are included for Modelling phase.

# 4. Modelling

Number of estimators is deployed in search for the the best model:
1. LogisticRegression
2. RandomForestClassifier
3. KNeighborsClassifier
4. SGDClassifier (Stochastic Gradient Descent)
5. GradientBoostingClassifier

Next, training sample performance is examinated.

#### Estimators are feeded with data binned with KBinsDiscretizer in an effort to counter outliers issue. All Data Preprocessing is conducted within pipelines using make_column_transformer procedure. Model is selected by GridSearchCV; adopted hyperparamteres grid is given in (Output 4.1).

In [ ]:
# Output 4.1
hypergrid = {
    'logreg': {
        'logisticregression__C':[1.4, 2, 2.6, 4, 5.7, 7.8],
        'logisticregression__solver':['lbfgs', 'liblinear']
    },
    'forest': {
        'randomforestclassifier__n_estimators':[8, 20, 22, 24, 46, 52],
        'randomforestclassifier__criterion':['gini', 'entropy'],
        'randomforestclassifier__min_samples_leaf':[16, 18, 19, 20]
    },
    'knn': {
        'kneighborsclassifier__n_neighbors':list(range(160, 220, 5)),
        'kneighborsclassifier__weights':['uniform', 'distance']
    },
    'sgd': {
        'sgdclassifier__loss':['modified_huber', 'squared_hinge', 'perceptron'],
        'sgdclassifier__penalty':['l2', 'l1', 'elasticnet'],
        'sgdclassifier__alpha':[0.0001, 0.001, 0.01]
    },

    'gbc': {
        'gradientboostingclassifier__learning_rate':[0.02, 0.075, 0.1, 0.15, 0.25],
        'gradientboostingclassifier__n_estimators':[24, 30, 60, 90, 120],
        'gradientboostingclassifier__loss':['exponential', 'deviance']
    }
}

#### Training is processed (Output 4.2).

In [ ]:
print('Output 4.2')

prep = make_column_transformer((KBinsDiscretizer(n_bins=10, encode='ordinal'), X_train.columns))

pipelines = {
    'logreg':make_pipeline(prep, LogisticRegression(random_state=8, max_iter=775)),
    'forest':make_pipeline(prep, RandomForestClassifier(random_state=8)),
    'knn':make_pipeline(prep, KNeighborsClassifier()),
    'sgd':make_pipeline(prep, SGDClassifier(random_state=8, max_iter=87000)),
    'gbc':make_pipeline(prep, GradientBoostingClassifier(random_state=8))   
}

fit_models_acc = {}
fit_models_auc = {}

for algo, pipeline in pipelines.items():
    model = GridSearchCV(pipeline, hypergrid[algo], cv=10, scoring='accuracy')
    model_auc = GridSearchCV(pipeline, hypergrid[algo], cv=10, scoring='roc_auc')
    try:
        print('Starting training for {}.'.format(algo))
        model.fit(X_train,y_train)
        model_auc.fit(X_train,y_train)
        fit_models_acc[algo] = model
        fit_models_auc[algo] = model_auc
        print('{} has been successfully fit.'.format(algo))
    except NotFittedError as e:
        print(repr(e))

#### Training performance is summarized in (Output 4.3)

In [ ]:
print('Output 4.3')
print('Title: in-sample training performance summary\n')

acc_train_results= {}
auc_train_results= {}

for algo in fit_models_acc.keys():
    acc_train_results[algo] = round(fit_models_acc[algo].best_score_, 5)
    auc_train_results[algo] = round(fit_models_auc[algo].best_score_ , 5)

acc_train_results = pd.Series(acc_train_results)
auc_train_results = pd.Series(auc_train_results)

train_results = pd.DataFrame(columns = ['ACC', 'AUC'])
train_results['ACC'] = acc_train_results
train_results['AUC'] = auc_train_results
print(train_results.sort_values(by='ACC', ascending=False))

# 5. Model Evaluation

In this section, estimators performance is evaluated on out-of-sample data. Metrics like Accuracy and AOC are computed. Moreover, confusion matrix is investigated and Sensitivity and Specificity is calculated. Last but not least, ROC curve is being examined. 

In [ ]:
print('Output 5.1')
print('Title: prediction performance summary\n')

acc_test_results= {}
auc_test_results= {}

for algo in fit_models_acc.keys():
    y_pred_acc = fit_models_acc[algo].predict(X_test)
    y_pred_auc = fit_models_auc[algo].predict(X_test)
    acc_test_results[algo] = round(accuracy_score(y_test, y_pred_acc), 5)
    auc_test_results[algo] = round(roc_auc_score(y_test, y_pred_auc), 5)

acc_test_results = pd.Series(acc_test_results)
auc_test_results = pd.Series(auc_test_results)

test_results = pd.DataFrame(columns = ['ACC', 'AUC'])
test_results['ACC'] = acc_test_results
test_results['AUC'] = auc_test_results
print(test_results.sort_values(by='ACC', ascending=False))

#### Confusion matrix for best estimator, logistic regression, is shown by (Output 5.2).

In [ ]:
print('Output 5.2')
print('Title: logreg confusion matrix\n')

y_pred = fit_models_acc['logreg'].predict(X_test)
confusion = confusion_matrix(y_test, y_pred)
print(confusion)

#### Sensitivity and Specificity are further calculated in order to investigate within target class prediction performance (Output 5.3).<br> Also, ROC curve is plotted (Output 5.4).

In [ ]:
print('Output 5.3')
print('Title: Prediction performance for logreg (Sensivity and Specificity)\n------')

TP = confusion[1,1]
TN = confusion[0,0]
FP = confusion[0,1]
FN = confusion[1,0]

# inspection of model performance within target distributions
## Sensitivity
print('Sensitivity, question inspected:')
print('When the acutal value is positive, how often is the prediction correct?')
print('sensitivity: {}\n'.format(round(TP/float(TP + FN), 4)))
## Specificity
print('Specificity, question inspected:')
print('When the acutal value is negative, how often is the prediction correct?')
print('specificity: {}'.format(round(TN/float(TN + FP), 4)))

In [ ]:
print('Output 5.4')
print('Receiver Operating Characteristic (logreg)\n')

proba = fit_models_acc['logreg'].predict_proba(X_test)
pred = proba[:,1]
fpr, tpr, thresholds = metrics.roc_curve(y_test, pred)
plt.figure(figsize=(12, 8))
plt.plot(fpr,tpr)
plt.axhline(y = round(TP/float(TP + FN), 5), color='r', linestyle='dotted') # default threshold = 0.5
plt.axvline(x = round(1 - TN/float(TN + FP), 5), color='r', linestyle='dotted') # # default threshold = 0.5
plt.axhline(y = 0.923728813559322, color='g', linestyle='dotted') # threshold = 0.5598
plt.axvline(x = 1-0.7931034482758621, color='g', linestyle='dotted') # # threshold = 0.5598
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('FPR (1-Specificity)')
plt.ylabel('TPR (Sensitivity)')
plt.grid(True)
plt.show()

#### According to ROC curve, there is opportunity to tune slightly up default threshold (0.5) in order to move to the point indicated by green dotted lines (Output 5.4). This process is done manually, though is not time consuming. Optimal threshold is 0.5598 and little Sensitivity is traded for a little Specificity, as (Output 5.5) shows. 

In [ ]:
print('Output 5.5\n')

def evaluate_threshold(threshold):
    print(tpr[thresholds > threshold][-1]) 
    print(1-fpr[thresholds > threshold][-1]) 

print('Optimal threshold (0.5598), sensitivity/specificity:')
evaluate_threshold(0.5598)
print('\n')
print('Threshold bigger by 0.0001 (0.5599), sensitivity/specificity:')
evaluate_threshold(0.5599)

#### Finally, the best model is (Output 5.6):

In [ ]:
print('Output 5.6')
print('Best estimator found for this data:\n\nLogistic regoression (threshold = 0.5598)')

new_thres_pred_acc = (fit_models_acc['logreg'].predict_proba(X_test)[:,1] >= 0.5598).astype(int)
new_thres_pred_auc = (fit_models_auc['logreg'].predict_proba(X_test)[:,1] >= 0.5598).astype(int)

print('ACC: {}'.format(round(accuracy_score(y_test, new_thres_pred_acc), 3)))
print(fit_models_acc['logreg'].best_params_)
print('AUC: {}'.format(round(roc_auc_score(y_test, new_thres_pred_auc), 3)))
print(fit_models_auc['logreg'].best_params_)